In [9]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import config

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Client
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter()
        self.twitter_client = API(self.auth)
        self.twitter_user  =twitter_user
    
    def get_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_frds(self, num_frds):
        frds = []
        for tweet in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_frds):
            frds.append(tweet)
        return frds



# Authenticator
class TwitterAuthenticator():
    def authenticate_twitter(self):
        auth = OAuthHandler(config.API_KEY, config.API_SECRET_KEY)
        auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)
        return auth     


# Streamer
class TwitterStreamer():
    """
    class for streaming and processing live tweets
    """
    def __init__(self):
        self.twitter_auth = TwitterAuthenticator()
        
    def stream_tweets(self, hashtags_list):
        # Handles twitter authentication and connection to twitter streamer API
        listener = TwitterListener()
        auth = self.twitter_auth.authenticate_twitter()
        stream = Stream(auth, listener)
        stream.filter(track=hashtags_list)
        

# Streamer Listener
class TwitterListener(StreamListener):
    """
    Basic listener class that prints to cmd
    """
    def on_data(self, data):
        print(data)
        return True
    
    def on_error(self, status):
        # Return False in case rate limit occurs
        if status==402:
            return False
        print(status)

In [23]:
if __name__ == "__main__":
    hashtags_list = ["donaldtrump"]
#     twitter_streamer = TwitterStreamer()
#     twitter_streamer.stream_tweets(hashtags_list)
    twitter_client = TwitterClient("realdonaldtrump")
    tweets = twitter_client.get_tweets(10)
    for t in tweets:
        print(t.text)
        print("\n\n")
    

RT @TeamTrump: Ivanka Trump: "America doesn’t need another empty vessel who will do whatever the media and the fringe of his party demands.…



https://t.co/XjI0JK0zXX



https://t.co/x6vlVTtc9N



https://t.co/WP1zsTCNkK



https://t.co/6bv45IRCye



RT @TeamTrump: Ivanka Trump: Donald Trump did not come to Washington to win praise from the beltway elites, he came to Washington to MAKE A…



RT @TeamTrump: Ivanka Trump: To those affected by Hurricane Laura, President Trump will support you every step of the way

#RNC2020 https:/…



Success: Since the National Guard moved into Kenosha, Wisconsin, two days ago, there has been NO FURTHER VIOLENCE,… https://t.co/eN6IGHuEA1



RT @realDonaldTrump: https://t.co/Ix0L4hF3LP



Great Ratings &amp; Reviews Last Night. Thank you!





In [36]:
class TweetAnalysizer():
    
    def tweets2df(self, tweets):
        df = pd.DataFrame(data = [tweet.text for tweet in tweets], columns=["Tweets"])
        df["id"] = np.array([tweet.id for tweet in tweets])
        df["geo"] = np.array([tweet.geo for tweet in tweets])
        df["date"] = np.array([tweet.created_at for tweet in tweets])
        
        return df

In [37]:
df = TweetAnalysizer().tweets2df(tweets)
df

,Tweets,id,geo,date
0,"RT @TeamTrump: Ivanka Trump: ""America doesn’t ...",1299404771095388160,None,2020-08-28 17:53:33
1,https://t.co/XjI0JK0zXX,1299404747099774978,None,2020-08-28 17:53:28
2,https://t.co/x6vlVTtc9N,1299404704175316992,None,2020-08-28 17:53:17
3,https://t.co/WP1zsTCNkK,1299404672999084033,None,2020-08-28 17:53:10
4,https://t.co/6bv45IRCye,1299404640983941121,None,2020-08-28 17:53:02
5,RT @TeamTrump: Ivanka Trump: Donald Trump did ...,1299404611976155137,None,2020-08-28 17:52:55
6,RT @TeamTrump: Ivanka Trump: To those affected...,1299404593881903105,None,2020-08-28 17:52:51
7,Success: Since the National Guard moved into K...,1299391572384059397,None,2020-08-28 17:01:07
8,RT @realDonaldTrump: https://t.co/Ix0L4hF3LP,1299347336997597184,None,2020-08-28 14:05:20
9,Great Ratings &amp; Reviews Last Night. Thank ...,1299345147294109697,None,2020-08-28 13:56:38
